### Considerações Iniciais

Este notebook é uma tradução do notebook [Cartographic Visualization](https://jjallaire.github.io/visualization-curriculum/altair_cartographic.html#geoshape-marks) do Visualization Curriculum. Devido às atualizações do Altair desde a publicação do notebook original, decidimos alterar algumas partes do texto e do código para mantê-los atualizados e coerentes com a versão atual do Altair (5.5.0).

# <font color="#747a7f">7</font> Visualização Cartográfica

*“A elaboração de mapas é uma das atividades intelectuais mais antigas da humanidade e também uma das mais complexas, com teoria científica, representação gráfica, fatos geográficos e considerações práticas combinados de maneiras variadas e incessantes.”* &mdash; [H. J. Steward](https://books.google.com/books?id=cVy1Ms43fFYC)

A Cartografia &ndash; o estudo e a prática de fazer mapas &ndash; tem uma rica história que abrange séculos de descobertas e design. A visualização cartográfica utiliza técnicas de mapeamento para transmitir dados contendo informações espaciais, como locais, rotas ou trajetórias na superfície da Terra.

<img width="300px" align="right" src="https://gist.githubusercontent.com/jheer/c90d582ef5322582cf4960ec7689f6f6/raw/8dc92382a837ccc34c076f4ce7dd864e7893324a/latlon.png"/>

Aproximando a Terra como uma esfera, podemos denotar posições usando um sistema de coordenadas esféricas de *latitude* (ângulo em graus ao norte ou ao sul do *equador*) e *longitude* (ângulo em graus especificando a posição leste-oeste). Neste sistema, um *paralelo* é um círculo de latitude constante e um *meridiano* é um círculo de longitude constante. O [*Meridiano de Greenwich*](https://en.wikipedia.org/wiki/Prime_meridian) está a 0° de longitude e, por convenção, é definido para passar pelo Observatório Real em Greenwich, Inglaterra.

Para "achatar" uma esfera tridimensional em um plano bidimensional, devemos aplicar uma [projeção](https://en.wikipedia.org/wiki/Map_projection) que mapeia pares (`longitude`, `latitude`) para coordenadas (`x`, `y`). Semelhante a [escalas](https://github.com/uwdata/visualization-curriculum/blob/master/altair_scales_axes_legends.ipynb), as projeções mapeiam de um domínio de dados (posição espacial) para um intervalo visual (posição do pixel). No entanto, os mapeamentos de escala que vimos até agora aceitam um domínio unidimensional, enquanto as projeções de mapa são inerentemente bidimensionais.

Neste notebook, vamos apresentar o básico sobre criação de mapas e visualizar dados espaciais com Altair, incluindo:

- Formatos de dados para representar características geográficas,
- Técnicas de geo-visualização, como mapas de pontos, símbolos e coropléticos, e
- Uma revisão das projeções cartográficas comuns.

*Este notebook é parte do [data visualization curriculum](https://github.com/uwdata/visualization-curriculum).*


In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

## <font color="#747a7f">7.1</font> Dados Geográficos: GeoJSON e TopoJSON


Até este ponto, nós trabalhamos com datasets formatados em JSON e CSV que correspondem a tabelas de dados compostas por linhas (registros) e colunas (campos). Para representar regiões geográficas (países, estados, *etc.*) e trajetórias (rotas de voo, linhas de metrô, *etc.*), nós precisamos expandir nosso repertório com formatos adicionais projetados para suportar geometrias ricas.

[GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) modela características geográficas dentro de um formato JSON especializado. Um `feature` do GeoJSON pode incluir dados geométricos &ndash; como coordenadas `longitude`, `latitude` que compõem a fronteira de um país &ndash; além de atributos de dados adicionais.

Aqui está um objeto `feature` do GeoJSON para a fronteira do estado do Colorado, nos EUA:


~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

A `feature` inclui um objeto `properties`, que pode conter qualquer número de campos, além de um objeto `geometry`, que neste caso contém um único polígono composto por coordenadas `[longitude, latitude]` para a fronteira do estado. As coordenadas continuam para a direita por um tempo, caso queira rolar...

Para saber mais sobre os detalhes técnicos do GeoJSON, consulte a [especificação oficial do GeoJSON](http://geojson.org/) ou leia o [guia útil de Tom MacWright](https://macwright.org/2015/03/23/geojson-second-bite).


Uma desvantagem do GeoJSON como formato de armazenamento é que ele pode ser redundante, resultando em tamanhos de arquivos maiores. Considere: o Colorado faz fronteira com outros seis estados (sete se incluir o canto que toca o Arizona). Ao invés de usar listas de coordenadas separadas, sobrepondo-se para cada um desses estados, uma abordagem mais compacta seria codificar as fronteiras compartilhadas apenas uma vez, representando a *topologia* das regiões geográficas. Felizmente, é exatamente isso que o formato [TopoJSON](https://github.com/topojson/topojson/blob/master/README.md) faz!


Vamos carregar um arquivo TopoJSON dos países do mundo (com resolução de 110 metros):

In [2]:
world = data.world_110m.url
world

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'

In [3]:
world_topo = data.world_110m()

In [4]:
world_topo.keys()

dict_keys(['type', 'transform', 'objects', 'arcs'])

In [5]:
world_topo['type']

'Topology'

In [6]:
world_topo['objects'].keys()

dict_keys(['land', 'countries'])

*Inspecione o objeto dicionário TopoJSON `world_topo` acima para ver seu conteúdo.*

Nos dados acima, a propriedade `objects` indica os elementos nomeados que nós podemos extrair dos dados: geometrias para todos os `countries` (países) ou um único polígono representando toda a `land` (terra) na Terra. Qualquer um destes pode ser desempacotado para dados GeoJSON que nós podemos então visualizar.

Como o TopoJSON é um formato especializado, precisamos instruir o Altair a analisar o formato TopoJSON, indicando qual objeto feature nomeado nós desejamos extrair da topologia. O seguinte código indica que nós queremos extrair as features GeoJSON do dataset `world` para o objeto `countries`:

~~~ js
alt.topo_feature(world, 'countries')
~~~

Este método `alt.topo_feature` se expande para o seguinte JSON do Vega-Lite:

~~~ json
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}
~~~

Agora que podemos carregar dados geográficos, nós estamos prontos para começar a fazer mapas!


## <font color="#747a7f">7.2</font> Marcas de Formas Geográficas

Para visualizar dados geográficos, o Altair oferece o tipo de marca `geoshape`. Para criar um mapa básico, nós podemos criar uma marca `geoshape` e passar nossos dados TopoJSON, os quais são então desempacotados em features GeoJSON, um para cada país do mundo:


In [7]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

No exemplo acima, o Altair aplica uma cor azul padrão e usa uma projeção de mapa padrão (`equalEarth`). Nós podemos personalizar as cores e as larguras das linhas de fronteira usando propriedades padrões da marca. Usando o método `project` nós também podemos adicionar nossa própria projeção de mapa:

In [8]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

Por padrão, o Altair ajusta automaticamente a projeção para que todos os dados se encaixem na largura e na altura do gráfico. Nós podemos também especificar parâmetros de projeção, como `scale` (nível de zoom) e `translate` (panning), para personalizar as configurações de projeção. Aqui nós ajustamos os parâmetros `scale` e `translate` para focar na Europa:

In [9]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

alt.Chart(...)

*Observe como a resolução de 110m dos dados se torna aparente nesta escala. Para ver contornos da costa e fronteiras mais detalhados, nós precisamos de um arquivo de entrada com geometrias mais refinadas. Ajuste os parâmetros `scale` e `translate` para focar o mapa em outras regiões!*



Até agora, nosso mapa mostra apenas países. Usando o operador `layer`, nós podemos combinar vários elementos de mapa. O Altair inclui *geradores de dados* que nós podemos usar para criar dados para camadas de mapa adicionais:

- O gerador de esfera (`{'sphere': True}`) oferece uma representação GeoJSON da esfera completa da terra. Nós podemos criar uma marca `geoshape` adicional que preencha a forma da Terra como uma camada de fundo.
- O gerador de gratícula (`{'graticule': ...}`) cria uma feature GeoJSON representando uma gratícula: uma grade formada por linhas de latitude e longitude. A gratícula padrão possui meridianos e paralelos a cada 10° entre ±80° de latitude. Para as regiões polares, existem meridianos a cada 90°. Essas configurações podem ser personalizadas usando as propriedades `stepMinor` e `stepMajor`.

Vamos dispor em camadas as marcas esfera, gratícula e país em uma especificação de mapa reutilizável:

In [10]:
map = alt.layer(
    # usa a esfera da terra como a camada base
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # adiciona uma gratícula para linhas de referência geográfica
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # e então os países do mundo
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)

Nós podemos estender o mapa com uma projeção desejada e desenhar o resultado. Aqui aplicamos uma [Projeção Natural da Terra](https://en.wikipedia.org/wiki/Natural_Earth_projection). A camada de *esfera* oferece o fundo azul claro; a camada de *gratícula* oferece as linhas de referência geográficas brancas.

In [11]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)

## <font color="#747a7f">7.3</font> Mapas de Pontos

Além dos dados *geométricos* oferecidos pelos arquivos GeoJSON ou TopoJSON, muitos datasets tabulares incluem informações geográficas na forma de campos para coordenadas de `longitude` e `latitude`, ou referências a regiões geográficas, como nomes de países, nomes de estados, códigos postais, *etc.*, os quais podem ser mapeados para coordenadas usando um [serviço de geocodificação](https://en.wikipedia.org/wiki/Geocoding). Em alguns casos, os dados de localização são ricos o suficiente para que nós possamos ver padrões significativos projetados apenas pelos pontos de dados &mdash; nenhum mapa base é necessário!

Vejamos um conjunto de dados de códigos postais de 5 dígitos nos Estados Unidos, incluindo coordenadas de `longitude` e `latitude` para cada agência dos correios, além de um campo `zip_code`.

In [12]:
zipcodes = data.zipcodes.url
zipcodes

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/zipcodes.csv'

Nós podemos visualizar a localização de cada agência dos correios usando uma pequena marca `square` (1 pixel). No entanto, para definir as posições, nós *não* usamos canais `x` e `y`. Por quê?

Embora as projeções cartográficas mapeiem coordenadas (`longitude`, `latitude`) para coordenadas (`x`,`y`), elas podem fazê-lo de maneiras arbitrárias. Não há garantia, por exemplo, que `longitude` → `x` e `latitude` → `y`!
Ao invés disso, o Altair inclui canais de codificação especiais de `longitude` e `latitude` para lidar com coordenadas geográficas. Estes canais indicam quais campos de dados devem ser mapeados para coordenadas de `longitude` e `latitude`, e em seguida aplica uma projeção para mapear essas coordenadas para posições (`x`, `y`).

In [13]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # aplica o campo chamado 'longitude' no canal longitude
    latitude='latitude:Q'    # aplica o campo chamado 'latitude' no canal latitude
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Ao plotar apenas os códigos postais, nós podemos ver o contorno dos Estados Unidos e discernir padrões significativos na densidade de agências dos correios, sem a necessidade de um mapa base ou elementos de referência adicionais!*

Nós usamos a projeção `albersUsa`, ao qual faz algumas adaptações na geometria real da Terra, com versões escalonadas do Alasca e do Havai no canto inferior esquerdo. Como não especificamos parâmetros de projeção `scale` ou `translate`, o Altair os define automaticamente para se ajustar aos dados visualizados.

Nós podemos agora seguir em frente e fazer mais perguntas sobre nosso dataset. Por exemplo, existe algum padrão na alocação dos códigos postais? Para avaliar esta questão, nós podemos adicionar uma codificação de cor com base no primeiro dígito do código postal. Primeiro, adicionamos uma transformação `calculate` para extrair o primeiro dígito e codificamos o resultado usando o canal de cor:

In [14]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('digit:N',title='dígito')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Para dar zoom em um dígito específico, adicione uma transformação de filtro para limitar os dados exibidos! Tente adicionar uma [seleção interativa](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) para filtrar por um único dígito e atualizar dinamicamente o mapa. E lembre-se de usar strings (\`'1'\`) em vez de números (\`1\`) ao filtrar os valores dos dígitos!_

(Este exemplo é inspirado pela clássica visualização [zipdecode](https://benfry.com/zipdecode/) de Ben Fry!)

Nós poderíamos ainda nos perguntar o que a *sequência* dos códigos postais poderia indicar. Uma forma de explorar esta questão é conectar cada código postal consecutivo usando uma marca `line`, como feito na visualização [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states) de Robert Kosara:

In [15]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Nós podemos agora ver como os códigos postais se agrupam em áreas menores, indicando uma alocação hierárquica dos códigos por local, mas com uma variação notável dentro dos grupos locais.*

Se você estava prestando atenção aos nossos mapas anteriores, talvez você possa ter notado que há códigos postais sendo plotados no canto superior esquerdo! Estes correspondem a locais como Porto Rico ou Samoa Americana, os quais contêm códigos postais dos EUA, mas são mapeados para coordenadas `null` (`0`, `0`) pela projeção `albersUsa`. Além disso, Alasca e Havai podem complicar nossa visão dos segmentos de linha conectados. Em resposta, o código acima inclui um filtro adicional que remove pontos fora dos intervalos de `longitude` e `latitude` escolhidos.

*Remova o filtro acima para ver o que acontece!*

## <font color="#747a7f">7.4</font> Mapas de Símbolos

Agora, vamos combinar um mapa base e os dados plotados como camadas separadas. Vamos examinar a rede de voos comerciais dos EUA, considerando tanto os aeroportos quanto as rotas de voo. Para isso, nós precisaremos de três datasets.
Para o nosso mapa base, nós usaremos um arquivo TopoJSON dos Estados Unidos com resolução de 10m, contendo features para `states` ou `counties`:

In [16]:
usa = data.us_10m.url
usa

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'

Para os aeroportos, nós usaremos um dataset com campos para as coordenadas de `longitude` e `latitude` de cada aeroporto, bem como o código do aeroporto `iata` &mdash; por exemplo, `'SEA'` para [Seattle-Tacoma International Airport](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport).

In [17]:
airports = data.airports.url
airports

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/airports.csv'

Por fim, nós utilizaremos um dataset de rotas de voo, que contém os campos `origin` e `destination` com os códigos IATA dos aeroportos correspondentes:

In [18]:
flights = data.flights_airport.url
flights

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/flights-airport.csv'

Vamos começar criando um mapa base usando a projeção `albersUsa` e adicionar uma camada que traça marcas `circle` para cada aeroporto:

In [19]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_São muitos aeroportos! Obviamente, nem todos são grandes hubs._

Semelhante ao nosso dataset de códigos postais, os dados dos aeroportos incluem pontos que estão fora dos Estados Unidos continentais. Então, novamente nós vemos pontos no canto superior esquerdo. Nós poderíamos querer filtrar estes pontos, mas para isso, nós precisamos saber mais sobre eles.

*Atualize a projeção do mapa acima para `albers` &ndash; contornando o comportamento peculiar do `albersUsa` &ndash; para que as localizações reais desses pontos adicionais sejam reveladas!*

Agora, em vez de mostrar todos os aeroportos de maneira indistinta, vamos identificar os grandes hubs considerando o número total de rotas que se originam em cada aeroporto. Nós usaremos o dataset `routes` como nossa principal fonte de dados: ele contém uma lista de rotas de voo que nós podemos agregar para contar o número de rotas para cada aeroporto de `origin`.

Entretanto, o dataset `routes` não inclui as *localizações* dos aeroportos! Para complementar os dados de `routes` com as localizações, nós precisamos de uma nova transformação de dados: `lookup`. A transformação `lookup` pega o valor de um campo em um dataset principal e o usa como uma *chave* para procurar informações relacionadas em outra tabela. Neste caso, nós queremos comparar o código do aeroporto de `origin` no nosso dataset `routes` com o campo `iata` do dataset `airports` e, em seguida, extrair os campos correspondentes de `latitude` e `longitude`.

In [20]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

*Quais aeroportos dos EUA têm o maior número de rotas de saída?*

Agora que nós podemos ver os aeroportos, talvez queiramos interagir com eles para entender melhor a estrutura da rede de tráfego aéreo. Nós podemos adicionar uma camada de marca `rule` para representar os caminhos de aeroportos de `origin` para aeroportos de `destination`, o que requer duas transformações `lookup` para recuperar as coordenadas de cada ponto final. Além disso, nós podemos usar uma seleção `point` para filtrar estas rotas, de modo que apenas as rotas que se originam no aeroporto atualmente selecionado sejam exibidas.

*Partindo do mapa estático acima, você consegue construir uma versão interativa? Fique à vontade para pular o código abaixo para interagir com o mapa e pensar em como poderia construí-lo por conta própria!*


In [21]:
# seleção interativa para o aeroporto de origem
# seleciona o aeroporto mais próximo ao cursor do mouse
origin = alt.selection_point(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# referência de dados compartilhada para transformações de pesquisa
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])

alt.layer(
    # mapa base dos Estados Unidos
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # linhas de rota do aeroporto de origem selecionado para aeroportos de destino
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filtro para origem selecionada apenas
    ).transform_lookup(
        lookup='origin', from_=foreign # origem lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # destino lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # tamanho dos Aeroportos por número de rotas de saída
    # 1. agrega o dataset flights-airport
    # 2. pesquisa os dados de localização do dataset dos aeroportos
    # 3. remove Porto Rico (PR) e Ilhas Virgens (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_params(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # círculos menores no topo
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

*Passe o mouse sobre o mapa para explorar a rede de voos!*

## <font color="#747a7f">7.5</font> Mapas coropléticos

Um [mapa coroplético](https://pt.wikipedia.org/wiki/Mapa_coropl%C3%A9tico) usa regiões sombreadas ou texturizadas para visualizar valores de dados. Mapas de símbolos dimensionados costumam ser mais precisos para ler, pois as pessoas tendem a ser melhores em estimar diferenças proporcionais entre a área de círculos do que entre tons de cores. No entanto, os mapas coropléticos são populares na prática e particularmente úteis quando muitos símbolos se tornam perceptivelmente esmagadores.

Por exemplo, embora os Estados Unidos tenham apenas 50 estados, eles têm milhares de condados dentro desses estados. Vamos construir um mapa coroplético da taxa de desemprego por condado, no ano de recessão de 2008. Em alguns casos, os arquivos GeoJSON ou TopoJSON de entrada podem incluir dados estatísticos que podemos visualizar diretamente. Neste caso, entretanto, nós temos dois arquivos: nosso arquivo TopoJSON que inclui features de fronteira de condado (`usa`), e um arquivo de texto separado que contém estatísticas de desemprego:

In [22]:
unemp = data.unemployment.url
unemp

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/unemployment.tsv'

Para integrar nossas fontes de dados, nós precisaremos usar novamente a transformação de `lookup`, aumentando nossos dados de `geoshape` baseados em TopoJSON com taxas de desemprego. Podemos então criar um mapa que inclua uma codificação de `color` para o campo `rate` pesquisado.

In [23]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True),
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Examine as taxas de desemprego por condado. Valores mais altos em Michigan podem estar relacionados à indústria automotiva. Condados nos estados das [Grandes Planícies](https://pt.wikipedia.org/wiki/Grandes_Plan%C3%ADcies) e Montanhas apresentam taxas baixas **e** altas. Esta variação é significativa ou é possivelmente um [artefato de tamanhos de amostra menores](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar mais, tente alterar o domínio de escala superior (por exemplo, para ``0,2``) para ajustar o mapeamento de cores.*

Uma preocupação central para mapas coropléticos é a escolha de cores. Acima, nós usamos o esquema padrão `'yellowgreenblue'` do Altair para mapas de calor. Abaixo, nós comparamos outros esquemas, incluindo um esquema *sequencial de matiz único* (`teals`) que varia apenas em luminância, um esquema *sequencial de vários matizes* (`viridis`) que aumenta em luminância e matiz e um esquema *divergente* (`blueorange`) que usa um ponto médio branco:

In [24]:
# função utilitária para gerar uma especificação de mapa para um esquema de cores oferecido
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

alt.HConcatChart(...)

*Quais esquemas de cores você acha mais eficazes? Por que poderia ser? Modifique os mapas acima para usar outros esquemas disponíveis, conforme descrito na documentação [Vega Color Schemes](https://vega.github.io/vega/docs/schemes/).*

## <font color="#747a7f">7.6</font> Projeções Cartográficas

Agora que nós temos alguma experiência na criação de mapas, vamos dar uma olhada mais de perto nas projeções cartográficas. Conforme explicado pela [Wikipedia](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_cartogr%C3%A1fica),

> Todas as projeções de mapa necessariamente distorcem a superfície de alguma forma. Dependendo do propósito do mapa, algumas distorções são aceitáveis ​​e outras não; portanto, diferentes projeções de mapa existem para preservar algumas propriedades do corpo esférico em detrimento de outras propriedades.



Algumas das propriedades que podemos querer considerar incluem:

* *Área:* A projeção distorce os tamanhos das regiões?

* *Orientação:* Uma linha reta corresponde a uma direção constante de viagem?

* *Distância:* Linhas de comprimento igual correspondem a distâncias iguais no globo?

* *Forma:* A projeção preserva relações espaciais (ângulos) entre pontos?

A seleção de uma projeção apropriada depende, portanto, do caso de uso do mapa. Por exemplo, se estivermos a avaliar o uso da terra e a extensão das questões fundiárias, poderemos escolher uma projecção de preservação da área. Se quisermos visualizar as ondas de choque que emanam de um terremoto, podemos focar o mapa no epicentro do terremoto e preservar as distâncias a partir desse ponto. Ou, se quisermos ajudar a navegação, a preservação do rumo e da forma pode ser mais importante.

Também podemos caracterizar as projeções em termos da *superfície de projeção*. As projeções cilíndricas, por exemplo, projetam pontos superficiais da esfera em um cilindro circundante; o cilindro “desenrolado” fornece então o nosso mapa. Como descreveremos abaixo, podemos alternativamente projetar na superfície de um cone (projeções cônicas) ou diretamente em um plano plano (projeções azimutais).

*Vamos primeiro desenvolver nossa intuição interagindo com uma variedade de projeções! [Abra o caderno online Vega-Lite Cartographic Projections](https://observablehq.com/@vega/vega-lite-cartographic-projections). Use os controles naquela página para selecionar uma projeção e explorar parâmetros de projeção, como a ``escala`` (zoom) e a translação x/y (panning). Os controles de rotação ([guinada, inclinação, rolagem](https://pt.wikipedia.org/wiki/Eixos_do_avi%C3%A3o)) determinam a orientação do globo em relação à superfície que está sendo projetada.*

### <font color="#747a7f">7.6.1</font> Um tour por tipos específicos de projeção

**[Projeções cilíndricas](https://jjallaire.github.io/visualization-curriculum/altair_cartographic.html)** mapeiam a esfera em um cilindro circundante, então desenrolam o cilindro. Se o eixo principal do cilindro estiver orientado norte-sul, os meridianos são mapeados para linhas retas. [Projeções pseudocilíndricas](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representam um meridiano central como uma linha reta, com outros meridianos “curvando-se” para longe do centro.

In [25]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='Cilíndrica equidistante'),
    minimap.project(type='mercator').properties(title='Mercator'),
    minimap.project(type='transverseMercator').properties(title='Transversa de mercator'),
    minimap.project(type='naturalEarth1').properties(title='Natural da terra')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)



* [Cilíndrica equidistante](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_cil%C3%ADndrica_equidistante) (``equirectangular``): Escala valores de coordenadas ``lat``, ``lon`` diretamente.

* [Mercator](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_de_Mercator) (``mercator``): Projeta em um cilindro, usando ``lon`` diretamente, mas submetendo ``lat`` a uma transformação não linear. Linhas retas preservam orientações de bússola constantes ([linha de rumo](https://pt.wikipedia.org/wiki/Loxodromia)), tornando esta projeção bem adequada para navegação. No entanto, áreas no extremo norte ou sul podem ser bastante distorcidas.

* [Transversa de mercator](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_transversa_de_Mercator) (``transverseMercator``): Uma projeção mercator, mas com o cilindro delimitador girado para um eixo transversal. Enquanto a projeção Mercator padrão tem maior precisão ao longo do equador, a projeção Transversa de mercator é mais precisa ao longo do meridiano central.

* [Natural da terra](https://en.wikipedia.org/wiki/Natural_Earth_projection) (``naturalEarth1``): Uma projeção pseudocilíndrica projetada para mostrar toda a Terra em uma visão.


[Projeções cônicas](https://en.wikipedia.org/wiki/Map_projection#Conic) mapeiam a esfera em um cone e, então, desenrolam o cone no plano. Projeções cônicas são configuradas por dois *paralelos padrões*, que determinam onde o cone intercepta o globo.

In [26]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='Área cônica igual'),
    minimap.project(type='conicEquidistant').properties(title='Cônica equidistante'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='Cônica Conforme'),
    minimap.project(type='albers').properties(title='Albers'),
    minimap.project(type='albersUsa').properties(title='Albers EUA')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

* [Área cônica igual](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_de_Albers) (``conicEqualArea``): projeção cônica que preserva a área. A forma e a distância não são preservadas, mas são aproximadamente precisas dentro dos paralelos padrão.

* [Cônica equidistante](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_c%C3%B4nica_equidistante_meridiana) (``conicEquidistant``): projeção cônica que preserva a distância ao longo dos meridianos e paralelos padrão.

* [Cônica Conforme](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_c%C3%B4nica_conforme_de_Lambert) (``conicConformal``): projeção cônica que preserva a forma (ângulos locais), mas não a área ou a distância.

* [Albers](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_de_Albers) (``albers``): uma variante da projeção de área cônica igual com paralelos padrão otimizados para criar mapas dos Estados Unidos.

* [Albers EUA](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_de_Albers) (``albersUsa``): uma projeção híbrida para os 50 estados dos Estados Unidos da América. Esta projeção une três projeções de Albers com parâmetros diferentes para os EUA continentais, Alasca e Havaí.





[Projeções azimutais](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) mapeiam a esfera diretamente em um plano.

In [27]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='Área Azimutal Igual'),
    minimap.project(type='azimuthalEquidistant').properties(title='Azimutal Equidistante'),
    minimap.project(type='orthographic').properties(title='Ortográfico'),
    minimap.project(type='stereographic').properties(title='Estereográfico'),
    minimap.project(type='gnomonic').properties(title='Gnomônico')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)



*   [Área Azimutal Igual](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_azimutal_de_Lambert) (``azimuthalEqualArea``): Projeta com precisão a área em todas as partes do globo, mas não preserva a forma (ângulos locais).

*   [Azimutal Equidistante](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_azimutal_equidistante) (``azimuthalEquidistant``): Preserva a distância proporcional do centro de projeção para todos os outros pontos do globo.

* [Ortográfico](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_ortogr%C3%A1fica) (``orthographic``): Projeta um hemisfério visível em um plano distante. Corresponde aproximadamente a uma visão da Terra do espaço sideral.

* [Estereográfico](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_estereogr%C3%A1fica) (``stereographic``): Preserva a forma, mas não a área ou a distância.

* [Gnomônico](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_gnom%C3%B4nica) (``gnomonic``): Projeta a superfície da esfera diretamente em um plano tangente. [Grandes círculos](https://pt.wikipedia.org/wiki/C%C3%ADrculo_m%C3%A1ximo) ao redor da Terra são projetados em linhas retas, mostrando o caminho mais curto entre os pontos.



## <font color="#747a7f">7.7</font> Coda: Dados Geográficos de Disputa

Todos os exemplos acima são extraídos da coleção vega-datasets, incluindo dados geométricos (TopoJSON) e tabulares (aeroportos, taxas de desemprego). Um desafio comum para começar com a visualização geográfica é coletar os dados necessários para sua tarefa. Vários provedores de dados são abundantes, incluindo serviços como o [United States Geological Survey](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics) e o [U.S. Census Bureau](https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html).

Em muitos casos, você pode ter dados existentes com um componente geográfico, mas requer medidas ou geometria adicionais. Para ajudar você a começar, aqui está um fluxo de trabalho:

1.   Visite o [Natural Earth Data](https://www.naturalearthdata.com/downloads/) e navegue para selecionar dados para regiões e resoluções de interesse. Baixe o(s) arquivo(s) zip correspondente(s).
2.   Vá para o [MapShaper](https://mapshaper.org/) e solte o arquivo zip baixado na página. Revise os dados conforme desejado e, em seguida, "Exportar" os arquivos TopoJSON ou GeoJSON gerados.
3.   Carregue os dados exportados do MapShaper para uso com o Altair!





Claro, muitas outras ferramentas - tanto de código aberto quanto proprietárias - existem para trabalhar com dados geográficos. Para mais sobre geo-data wrangling e criação de mapas, veja a série de tutoriais de Mike Bostock sobre [Command-Line Cartography](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c).

## <font color="#747a7f">7.8</font> Sumário

Neste ponto, nós apenas mergulhamos os dedos dos pés nas águas da cartografia. (*Você não esperava que um único caderno transmitisse séculos de aprendizado, esperava?*) Por exemplo, deixamos intocados tópicos como [cartogramas](https://pt.wikipedia.org/wiki/Cartograma) e [topografia](https://pt.wikipedia.org/wiki/Topografia) de transmissão - como no livro esclarecedor de Imhof, [Cartographic Relief Presentation](https://books.google.com.br/books?id=cVy1Ms43fFYC&redir_esc=y). No entanto, agora você deve estar bem equipado para criar uma rica variedade de geovisualizações. Para mais, o livro de MacEachren [How Maps Work: Representation, Visualization, and Design](https://books.google.com.br/books?id=xhAvN3B0CkUC&redir_esc=y) fornece uma visão geral valiosa da cartografia da perspectiva da visualização de dados.